In [1]:
import pandas as pd
import pandas as pd
from datetime import datetime
import ast

In [2]:
resume = pd.read_csv('resume1.csv')
vacancy = pd.read_csv('vacancy1.csv')

In [3]:
vacancy = vacancy.drop_duplicates()
resume = resume.drop_duplicates()

In [4]:
def translate_date(date_str):
    if pd.notna(date_str):
        for ru_month, en_month in ru_to_en_month.items():
            if ru_month in date_str:
                date_str = date_str.replace(ru_month, en_month)
                break
    return date_str

In [5]:
ru_to_en_month = {
    'января': 'January',
    'февраля': 'February',
    'марта': 'March',
    'апреля': 'April',
    'мая': 'May',
    'июня': 'June',
    'июля': 'July',
    'августа': 'August',
    'сентября': 'September',
    'октября': 'October',
    'ноября': 'November',
    'декабря': 'December'
}
resume['birth_date'] = resume['birth_date'].apply(translate_date)
resume['birth_date'] = pd.to_datetime(resume['birth_date'])

In [6]:
resume['gender'] = resume['gender'].replace({'Мужчина': 0, 'Женщина': 1})

In [7]:
resume.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2883 entries, 0 to 4692
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   birth_date       2721 non-null   datetime64[ns]
 1   gender           2883 non-null   object        
 2   area             2883 non-null   object        
 3   title            2883 non-null   object        
 4   specialization   2883 non-null   object        
 5   salary           2883 non-null   object        
 6   education_level  2883 non-null   object        
 7   education        2883 non-null   object        
 8   language         2883 non-null   object        
 9   experience       2883 non-null   object        
 10  skill_set        2883 non-null   object        
 11  skills           2564 non-null   object        
dtypes: datetime64[ns](1), object(11)
memory usage: 292.8+ KB


In [9]:
for col in resume.columns:
    resume[col] = resume[col].replace('[', '').replace(']', '')

In [10]:
def extract_element(row, key):
    if isinstance(row, dict) and key in row:
        return row[key]
    else:
        return None

In [11]:
for i in resume.columns[4:11]:
    try:
        resume[i] = [ast.literal_eval(d) for d in resume[i]]
        if isinstance(resume[i][0], dict):
            for key in resume[i][0].keys():
                resume[i +'_'+ key] = resume[i].apply(lambda x: extract_element(x, key))
            resume.drop(i, axis=1, inplace=True)
    except Exception as e:
        continue

In [12]:
def extract_names(row):
    names = [item['name'] for item in row]
    return ', '.join(names)

resume['specialization_names'] = resume['specialization'].apply(extract_names)
resume.drop('specialization', axis=1, inplace=True)

In [13]:
resume.experience[4]

[{'start': '01-04-2021', 'end': None, 'position': '', 'description': 'ИТ'},
 {'start': '01-09-2003',
  'end': '01-11-2020',
  'position': '',
  'description': 'Полный спектр обязанностей в сфере IT-сопровождения деятельности государственной организации.'},
 {'start': '01-09-1999',
  'end': '01-09-2003',
  'position': '',
  'description': 'Администратор БД физ. лиц; общие вопросы IT-сопровождения деятельности организации'}]

In [14]:
for col in vacancy.columns:
    vacancy[col] = vacancy[col].replace('[', '').replace(']', '')

In [15]:
vacancy['professional_roles'][0]

"[{'id': '70', 'name': 'Менеджер по продажам, менеджер по работе с клиентами'}]"

In [16]:
import ast
import numpy as np
def extract_names_vac(i):
    a = []
    i = ast.literal_eval(i)
    for j in i:
        a.append(j['name'])
    return a
def extract(i):
    if i is not np.NaN:
        i = ast.literal_eval(i)
        return i['name']

In [17]:
def unpack_dict_column(i):
    if pd.isna(i):
        return None
    if isinstance(i, str):
        try:
            i = ast.literal_eval(i)
        except (ValueError, SyntaxError):
            return None
    if isinstance(i, dict) and 'raw' in i:
        return i['raw']
    return None
vacancy['address'] = vacancy['address'].apply(unpack_dict_column)

In [18]:
vacancy['experience'] = vacancy['experience'].apply(extract)
vacancy['area'] = vacancy['area'].apply(extract)
vacancy['type'] = vacancy['type'].apply(extract)
vacancy['employment'] = vacancy['employment'].apply(extract)
vacancy['employer'] = vacancy['employer'].apply(extract)
vacancy['schedule'] = vacancy['schedule'].apply(extract)
vacancy['billing_type'] = vacancy['billing_type'].apply(extract)
vacancy['department'] = vacancy['department'].apply(extract)


In [19]:
vacancy['key_skills'] = vacancy['key_skills'].apply(extract_names_vac)
vacancy['professional_roles'] = vacancy['professional_roles'].apply(extract_names_vac)
vacancy['specializations'] = vacancy['specializations'].apply(extract_names_vac)
vacancy['working_time_modes'] = vacancy['working_time_modes'].apply(extract_names_vac)
vacancy['relations'] = vacancy['relations'].apply(extract_names_vac)
vacancy['working_time_intervals'] = vacancy['working_time_intervals'].apply(extract_names_vac)
vacancy['working_days'] = vacancy['working_days'].apply(extract_names_vac)


In [20]:
try:
    vacancy['salary'] = [ast.literal_eval(d) for d in vacancy['salary']]
    if isinstance(vacancy['salary'][0], dict):
        for key in vacancy['salary'][0].keys():
            vacancy[i +'_'+ key] = vacancy['salary'].apply(lambda x: extract_element(x, key))
        vacancy.drop(i, axis=1, inplace=True)
except Exception as e:
    print('k')

k


In [21]:
vacancy['initial_created_at'] = pd.to_datetime(vacancy['initial_created_at'], utc=True)

In [22]:
import re
def clean_des(html_text):
    text = re.sub(r'\xa0', ' ', re.sub(r'\n', ' ', re.sub(r"<[^>]+>", "", html_text, flags=re.S)))
    text = ' '.join(text.split())
    return text

vacancy['description'] = vacancy['description'].apply(clean_des)

In [25]:
resume.to_csv('resume.csv', index=False)
vacancy.to_csv('vacancy.csv', index=False)

In [27]:
vacancy

,response_letter_required,working_time_modes,response_url,initial_created_at,billing_type,approved,code,alternate_url,specializations,suitable_resumes_url,...,description,vacancy_constructor_template,hidden,test,id,contacts,professional_roles,video_vacancy,key_skills,languages
0,False,[],NaN,2007-10-11 12:20:53+00:00,Стандарт,True,NaN,https://hh.ru/vacancy/998476,[],NaN,...,Обязанности: работа с имеющейся клиентской баз...,NaN,False,NaN,998476,NaN,"[Менеджер по продажам, менеджер по работе с кл...",NaN,[],[]
1,False,[],NaN,2024-05-03 07:29:04+00:00,Премиум,True,NaN,https://hh.ru/vacancy/98309142,[],NaN,...,Delko - один из лидеров грузоперевозок в Росси...,NaN,False,NaN,98309142,NaN,"[Менеджер по логистике, менеджер по ВЭД]",NaN,"[Транспортная логистика, Автомобильные грузопе...",[]
2,False,[],NaN,2024-05-03 06:44:14+00:00,Премиум,True,NaN,https://hh.ru/vacancy/98303074,[],NaN,...,Мы предлагаем: Официальное трудоустройство сог...,NaN,False,NaN,98303074,NaN,[Кладовщик],NaN,"[Работа в команде, Работа на складе, Высокая э...",[]
3,False,[],NaN,2024-04-01 05:56:00+00:00,Премиум,True,NaN,https://hh.ru/vacancy/95856059,[],NaN,...,Ищешь работу с возможностью карьерного развити...,NaN,False,NaN,95856059,NaN,"[Менеджер по продажам, менеджер по работе с кл...",NaN,[],[]
4,False,[],NaN,2024-05-02 06:09:47+00:00,Премиум,True,NaN,https://hh.ru/vacancy/98206193,[],NaN,...,ПОСТРОЙТЕ КАРЬЕРУ В ЭТМ! Мы приглашаем как опы...,NaN,False,NaN,98206193,NaN,"[Упаковщик, комплектовщик]",NaN,"[Складской Учет, Работа с ТМЦ, Точность и вним...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2044,False,[],https://hh.ru/article/31717?utm_source=hh_lead...,2024-04-03 09:32:04+00:00,Премиум,True,NaN,https://hh.ru/vacancy/96085191,[],NaN,...,Перед вами не просто вакансия. Перед вами — ун...,NaN,False,NaN,96085191,NaN,"[Оператор call-центра, специалист контактного ...",NaN,[],[]
2045,False,[],https://hh.ru/article/31717?utm_source=hh_lead...,2024-04-03 09:34:02+00:00,Премиум,True,NaN,https://hh.ru/vacancy/96085614,[],NaN,...,Перед вами не просто вакансия. Перед вами — ун...,NaN,False,NaN,96085614,NaN,"[Оператор call-центра, специалист контактного ...",NaN,[],[]
2046,False,[],https://hh.ru/article/31717?utm_source=hh_lead...,2024-04-03 09:35:14+00:00,Премиум,True,NaN,https://hh.ru/vacancy/96086001,[],NaN,...,Перед вами не просто вакансия. Перед вами — ун...,NaN,False,NaN,96086001,NaN,"[Оператор call-центра, специалист контактного ...",NaN,[],[]
2047,False,[],https://hh.ru/article/31717?utm_source=hh_lead...,2024-04-03 09:36:04+00:00,Премиум,True,NaN,https://hh.ru/vacancy/96086258,[],NaN,...,Перед вами не просто вакансия. Перед вами — ун...,NaN,False,NaN,96086258,NaN,"[Оператор call-центра, специалист контактного ...",NaN,[],[]
